In [1]:
import pandas as pd
import os
import numpy as np 

In [2]:
file_path = r'C:\Users\USER\Paper\with index_V3\\'
test = pd.DataFrame()
for i in range(0,1800,100):
    test_path = pd.read_csv(os.path.join(file_path, 'backtest_' + str(i) + '_' + str(i+100) + '.csv')).iloc[:,1:]
    test = pd.concat([test,test_path])

In [3]:
df_drop_test = test.drop_duplicates('code',keep='first').reset_index(drop=True)

In [4]:
list[test['predict_rate'].quantile(0.99),test['predict_rate'].quantile(0.01)]

list[0.9933777751999999, 0.0002091473752]

In [5]:
test

,code,Date,Decision,predict_rate,Open,High,Low,Close,RR
0,Y9999,2023-02-24,No Action,0.058359,15669.88,15717.96,15503.79,15503.79,0.0
1,Y9999,2023-03-01,No Action,0.034274,15435.38,15617.03,15387.38,15598.49,0.0
2,Y9999,2023-03-02,No Action,0.012953,15580.28,15618.06,15492.37,15598.72,0.0
3,Y9999,2023-03-03,No Action,0.005483,15621.36,15713.42,15606.85,15608.42,0.0
4,Y9999,2023-03-06,No Action,0.006265,15675.28,15818.88,15675.28,15763.51,0.0
...,...,...,...,...,...,...,...,...,...
16109,8929,2023-12-22,No Action,0.684511,20.35,20.35,20.30,20.30,0.0
16110,8929,2023-12-25,No Action,0.775693,20.30,20.40,20.25,20.40,0.0
16111,8929,2023-12-26,No Action,0.895652,20.30,20.40,20.30,20.30,0.0
16112,8929,2023-12-27,No Action,0.903871,20.30,20.30,20.10,20.30,0.0


In [6]:
backtest = pd.DataFrame()
upper_quantile = test['predict_rate'].quantile(0.98)
lower_quantile = test['predict_rate'].quantile(0.0015)

test['Decision'] = test['predict_rate'].apply(lambda x: 'Buy' if x >= upper_quantile else 'Sell' if x <= lower_quantile else 'No Action')
test['RR'] = 0

for code in df_drop_test['code']:
    test_df = test[test['code'] == code].reset_index(drop=True)
    
    if not test_df[test_df['Decision'] == 'Buy'].empty and not test_df[test_df['Decision'] == 'Sell'].empty:
        buy_closes = test_df[test_df['Decision'] == 'Buy']['Close']
        sell_closes = test_df[test_df['Decision'] == 'Sell']['Close']

        # Get all buy-sell pairs where buy_index < sell_index for Buy scenario
        valid_pairs_B = [(b, s) for b in buy_closes.index for s in sell_closes.index if b < s]
        # Get all buy-sell pairs where buy_index > sell_index for Sell scenario
        valid_pairs_S = [(b, s) for b in buy_closes.index for s in sell_closes.index if b > s]
    
        # Check for valid pairs for Buy
        if valid_pairs_B:
            avg_buy_close = buy_closes.mean()
            avg_sell_close = sell_closes.mean()

            # Calculate RR using average "Close" values
            rr_value_buy = (avg_sell_close / avg_buy_close) - 1
            test_df.loc[test_df[test_df['Decision'] == 'Buy'].index[0], 'RR'] = rr_value_buy

        # Check for valid pairs for Sell
        elif valid_pairs_S:
            avg_sell_close = sell_closes.mean()
            avg_buy_close = buy_closes.mean()

            # Calculate RR using average "Close" values
            rr_value_sell = (avg_sell_close / avg_buy_close) - 1
            test_df.loc[test_df[test_df['Decision'] == 'Sell'].index[0], 'RR'] = rr_value_sell

   
    backtest = pd.concat([backtest,test_df])

C:\Users\USER\AppData\Local\Temp\ipykernel_15496\3279167160.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2450917508735102' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_df.loc[test_df[test_df['Decision'] == 'Buy'].index[0], 'RR'] = rr_value_buy
C:\Users\USER\AppData\Local\Temp\ipykernel_15496\3279167160.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.21618924998033373' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_df.loc[test_df[test_df['Decision'] == 'Buy'].index[0], 'RR'] = rr_value_buy
C:\Users\USER\AppData\Local\Temp\ipykernel_15496\3279167160.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.08274941608274955' has dtype incompatible with int64, please e

C:\Users\USER\AppData\Local\Temp\ipykernel_15496\3279167160.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.06523768341497149' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_df.loc[test_df[test_df['Decision'] == 'Buy'].index[0], 'RR'] = rr_value_buy
C:\Users\USER\AppData\Local\Temp\ipykernel_15496\3279167160.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.1245302677532012' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_df.loc[test_df[test_df['Decision'] == 'Sell'].index[0], 'RR'] = rr_value_sell


In [7]:
'''
# Vol Change
backtest = pd.DataFrame()
upper_quantile = test['predict_rate'].quantile(0.97)
lower_quantile = test['predict_rate'].quantile(0.002)

test['Decision'] = test['predict_rate'].apply(lambda x: 'Buy' if x >= upper_quantile else 'Sell' if x <= lower_quantile else 'No Action')
test['Vol'] = 0
test['unrealized_pl'] = 0
test['realized_pl'] = 0
test['Avg_Price'] = 0
test['total_cost'] = 0

for code in df_drop_test['code']:
    test_df = test[test['code'] == code].reset_index(drop=True)
    
    # Initialize cumulative variables for tracking volume and total cost
    cumulative_vol = 0
    cumulative_total_cost = 0
    
    for idx, row in test_df.iterrows():
        price = test_df.loc[idx, "Close"]

            # Check the decision for the current row
        if test_df.loc[idx, "Decision"] == "Buy":  
                # Increment cumulative volume by 1 for each buy
            cumulative_vol += 1  
            cumulative_total_cost += price  # Add current price to cumulative total cost

        elif test_df.loc[idx, "Decision"] == "Sell":
                # Decrement cumulative volume by 1 for each sell
            cumulative_vol -= 1
            cumulative_total_cost -= price
        test_df.loc[idx, "Vol"] = cumulative_vol
        test_df.loc[idx, "total_cost"] = cumulative_total_cost
        test_df.loc[idx, "Avg_Price"] = cumulative_total_cost / cumulative_vol if cumulative_vol != 0 else 0
        test_df.loc[idx, 'unrealized_pl'] = test_df.loc[idx, "Vol"] * (price - test_df.loc[idx, "Avg_Price"])   
        test_df.loc[idx, 'RR'] = (test_df.loc[idx, 'realized_pl'] + test_df.loc[idx, 'unrealized_pl']) / abs(cumulative_total_cost)
        if idx > 0 and abs(test_df.loc[idx, "Vol"]) < abs(test_df.loc[idx-1, "Vol"]) and test_df.loc[idx-1, "Vol"] != 0 and cumulative_vol != 0:

            # Calculate the average price and unrealized profit/loss

            test_df.loc[idx, 'realized_pl'] = (test_df.loc[idx-1, "Vol"] - test_df.loc[idx, "Vol"]) * (price - test_df.loc[idx-1, "Avg_Price"])
            test_df.loc[idx, 'unrealized_pl'] = test_df.loc[idx, "Vol"] * (price - test_df.loc[idx, "Avg_Price"])
            test_df.loc[idx, 'unrealized_pl'] = test_df.loc[idx, 'unrealized_pl'] - test_df.loc[idx, 'realized_pl']
            test_df.loc[idx, 'RR'] = (test_df.loc[idx, 'realized_pl'] + test_df.loc[idx, 'unrealized_pl']) / abs(cumulative_total_cost) 
            
        elif idx > 0 and test_df.loc[idx-1, "Vol"] != 0 and cumulative_vol == 0:
            test_df.loc[idx, 'unrealized_pl'] = 0 
            test_df.loc[idx, 'realized_pl'] = test_df.loc[idx, "Vol"] * (price - test_df.loc[idx, "Avg_Price"])
            cumulative_total_cost = 0
            test_df.loc[idx, "total_cost"] = cumulative_total_cost        
            test_df.loc[idx, 'RR'] = (test_df.loc[idx, 'realized_pl'] + test_df.loc[idx, 'unrealized_pl']) / abs(cumulative_total_cost)
        

        # Append updated test_df to backtest
    backtest = pd.concat([backtest, test_df])
'''



'\n# Vol Change\nbacktest = pd.DataFrame()\nupper_quantile = test[\'predict_rate\'].quantile(0.97)\nlower_quantile = test[\'predict_rate\'].quantile(0.002)\n\ntest[\'Decision\'] = test[\'predict_rate\'].apply(lambda x: \'Buy\' if x >= upper_quantile else \'Sell\' if x <= lower_quantile else \'No Action\')\ntest[\'Vol\'] = 0\ntest[\'unrealized_pl\'] = 0\ntest[\'realized_pl\'] = 0\ntest[\'Avg_Price\'] = 0\ntest[\'total_cost\'] = 0\n\nfor code in df_drop_test[\'code\']:\n    test_df = test[test[\'code\'] == code].reset_index(drop=True)\n    \n    # Initialize cumulative variables for tracking volume and total cost\n    cumulative_vol = 0\n    cumulative_total_cost = 0\n    \n    for idx, row in test_df.iterrows():\n        price = test_df.loc[idx, "Close"]\n\n            # Check the decision for the current row\n        if test_df.loc[idx, "Decision"] == "Buy":  \n                # Increment cumulative volume by 1 for each buy\n            cumulative_vol += 1  \n            cumulative_tot

In [8]:
backtest.to_csv('backtest_all.csv')

In [9]:
test_df

,code,Date,Decision,predict_rate,Open,High,Low,Close,RR
0,8929,2023-02-24,No Action,0.209072,21.20,21.35,21.05,21.15,0
1,8929,2023-03-01,No Action,0.204093,21.15,22.30,21.15,21.80,0
2,8929,2023-03-02,No Action,0.158443,21.80,22.50,21.70,22.15,0
3,8929,2023-03-03,No Action,0.123636,22.15,22.40,22.15,22.20,0
4,8929,2023-03-06,No Action,0.047720,22.25,22.25,21.85,22.00,0
...,...,...,...,...,...,...,...,...,...
203,8929,2023-12-22,No Action,0.684511,20.35,20.35,20.30,20.30,0
204,8929,2023-12-25,No Action,0.775693,20.30,20.40,20.25,20.40,0
205,8929,2023-12-26,No Action,0.895652,20.30,20.40,20.30,20.30,0
206,8929,2023-12-27,No Action,0.903871,20.30,20.30,20.10,20.30,0
